<a href="https://colab.research.google.com/github/junsookim1/Everday_Practice/blob/master/Koyang_Bike_Data(csv%ED%8C%8C%EC%9D%BC_%EB%B0%8F_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%95%EB%A6%AC_%EA%B0%80%EC%9D%B4%EB%93%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import missingno as msno
from haversine import haversine

## 기본 자전거 데이터 
- 기존 운영이력 데이터에서 columns 이름만 한글로 변경
- 이동 시간(s) 구함 : 반납시간 - 대여

In [ ]:
df_OH = pd.read_csv('01.운영이력_ver1(한글컬럼, 이용거리).csv')
df_Bike = pd.read_csv('02.자전거스테이션.csv')

/home/jun/anaconda3/envs/jun/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_OH # 기존 운영이력 데이터에서 columns 이름만 한글로 변경 / # 이동시간(s)구함

,대여 번호,대여 시간,대여 스테이션 번호,대여 거치대 번호,반납 시간,반납 스테이션 번호,반납 거치대 번호,추정 이동 거리,회원 구분,회원 번호,비회원 번호,자전거 번호,관제 반납,이용 시간(s)
0,15945541,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,791.0
1,15945542,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,2961.0
2,15945543,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,3600.0
3,15945544,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,829.0
4,15945545,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,912.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142683,21718608,2019-12-31 23:52:39,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN,377.0
3142684,21718610,2019-12-31 23:53:59,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN,240.0
3142685,21718611,2019-12-31 23:55:27,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN,657.0
3142686,21718613,2019-12-31 23:59:14,264,4,2020-01-01 00:10:39,210,13,0.0,12,180671,0.0,1A84400000F804,NaN,685.0


In [ ]:
df_Bike

,Station_ID,STATION_NAME,거치대 수량,위도,경도
0,101,어울림마을 701동 앞,20,37.654775,126.834584
1,103,대림e-편한세상106동,20,37.660442,126.840377
2,104,탄현마을8단지,25,37.698523,126.766042
3,105,KT 덕양지사 앞,20,37.655244,126.839261
4,106,원당역 앞 공영주차장,30,37.653410,126.842530
...,...,...,...,...,...
159,350,★하이파크5단지 502동앞 버스정류장,20,37.697867,126.753089
160,351,◆일산에듀포레 푸르지오,40,37.702259,126.767231
161,352,◆꿈에그린203동앞,40,37.666425,126.749244
162,353,◆꿈에그린106동앞,40,37.666720,126.750784


## 스테이션 구분
### Koyang_Bike_Data(스테이션 구분, 회원 구분)에서 확인 
- 대여, 반납, 현재 스테이션의 unique값
- 각각의 unique값을 통해 스테이션 간의 차이점 분석
- 반납 스테이션 : 0,998,999 데이터

## 회원구분
### Koyang_Bike_Data(스테이션 구분, 회원 구분)에서 확인 
- 회원 / 비회원간의 자전거 이용시간, 이동거리 총 합
- 고양시 자전거를 이용하는 총 회원수 : 39127
- 고양시 자전거 비회원 이용 횟수 :792388 이용 횟수 

 ## 스테이션 거리(haversine) 기법
 ### Koyang_Bike_Data(스테이션 거리)
 - 각 스테이션의 이동거리 : 스테이션 별 이동거리.csv(load)
 - 대여 스테이션과 반납 스테이션이 같을 경우 추정 이동거리가 0과 0이 아닐때 이동 시간 데이터 분석
 #### 대여 스테이션 번호 == 반납 스테이션 번호
- 추정 이동거리가 0과 0이 아닌경우 구하고
- 그 중에서 이용시간이 0과 0이 아닌 경우 같이 구한다.
- df3 : 추정 이동거리가 != 0
- df4 : 추정 이동거리가 == 0
 

In [ ]:
df_sta_har = pd.read_csv('스테이션 별 이동거리.csv')
df_sta_har

,Unnamed: 0,sta_rent,sta_return,이동거리
0,0,101,101,0.000000
1,1,101,103,810.593473
2,2,101,104,7749.401651
3,3,101,105,414.953376
4,4,101,106,715.798911
...,...,...,...,...
26891,26891,992,350,9768.976190
26892,26892,992,351,9278.739241
26893,26893,992,352,8104.699800
26894,26894,992,353,7993.663782


## 이동거리,시간(월/일 단위)
### Koyang_Bike_Data(월 단위 구분_이동거리,시간)
### Koyang_Bike_Data(일 단위 구분_이동거리,시간)
- 각 스테이션의 이동거리,시간 : 월별 이동거리_시간(평균,합).csv(load)
- 각 스테이션의 이동거리,시간 : 일별 이동거리_시간(평균,합).csv(load)

In [ ]:
df_month_mean_sum = pd.read_csv('월별 이동거리_시간(평균,합).csv')
df_month_mean_sum 

,대여 시간,추정 이동 거리(평균),이용 시간(s)(평균),추정 이동 거리(합),이용 시간(s)(합)
0,2017-01-31,8903.751314,1496.286333,2.930848e+08,54969071.0
1,2017-02-28,8764.307791,1566.171357,2.807909e+08,55542701.0
2,2017-03-31,9163.443077,1602.969166,5.813746e+08,111927322.0
3,2017-04-30,9258.841761,1846.604920,9.356615e+08,204688769.0
4,2017-05-31,8744.256897,1696.929611,1.130536e+09,240186811.0
5,2017-06-30,8933.243546,1607.333249,1.217074e+09,240037540.0
6,2017-07-31,8494.810944,1520.677718,8.896700e+08,175493812.0
7,2017-08-31,7345.921713,1575.344774,8.213255e+08,193606722.0
8,2017-09-30,6057.658008,1627.275733,8.544145e+08,251339246.0
9,2017-10-31,4959.453382,1686.174327,6.180967e+08,230215067.0


In [ ]:
df_day_mean_sum = pd.read_csv('일별 이동거리_시간(평균,합).csv')
df_day_mean_sum

,대여 시간,추정 이동 거리(평균),이용 시간(s)(평균),추정 이동 거리(합),이용 시간(s)(합)
0,2017-01-01,9533.028766,1673.962457,12593131.0,2452355.0
1,2017-01-02,10088.323926,1435.165138,15727697.0,2502928.0
2,2017-01-03,9267.804818,1466.927812,15004576.0,2621400.0
3,2017-01-04,9712.572571,1365.047398,16997002.0,2649557.0
4,2017-01-05,9465.182827,1325.515328,16204393.0,2507875.0
...,...,...,...,...,...
1090,2019-12-27,2982.797459,1419.180889,4226624.0,2361517.0
1091,2019-12-28,3227.851692,1613.333754,4483486.0,2557134.0
1092,2019-12-29,3673.012311,1429.379338,3878701.0,1771001.0
1093,2019-12-30,2625.471115,1335.290098,3226704.0,1914806.0


## 스테이션 이용 횟수 (월/일 단위)
### Koyang_Bike_Data(월 단위 구분_(대여,반납)스테이션 이동 횟수)
#### 모든 기준은 대여시간 기준으로 작성하였습니다.
- 대여 스테이션 월별 이용 현황 : 월별 대여 스테이션 이용 현황(완성).csv(load)
- 대여 스테이션 총 이용 현황 : 총 대여 스테이션 이용 현황(완성).csv(load)
- 반납 스테이션 월별 이용 현황 : 월별 반납 스테이션 이용 현황(완성).csv(load)
- 반납 스테이션 총 이용 현황 : 총 반납 스테이션 이용 현황(완성).csv(load)


In [ ]:
df_ren_mon_sta = pd.read_csv('월별 대여 스테이션 이용 현황(완성).csv')
df_ren_to_sta = pd.read_csv('총 대여 스테이션 이용 현황(완성).csv')
df_return_mon_sta = pd.read_csv('월별 반납 스테이션 이용 현황(완성).csv')
df_return_to_sta = pd.read_csv('총 반납 스테이션 이용 현황(완성).csv')

In [ ]:
df_ren_mon_sta

,대여 시간(M),대여 스테이션 번호,이용 횟수(월)
0,2017-01,101,80
1,2017-01,103,33
2,2017-01,104,152
3,2017-01,105,27
4,2017-01,106,62
...,...,...,...
5611,2019-12,349,133
5612,2019-12,350,54
5613,2019-12,351,157
5614,2019-12,992,19


In [ ]:
df_ren_to_sta

,Unnamed: 0,이용 횟수(전체),이용 횟수(전체).1
0,0,101,8366
1,1,103,4535
2,2,104,13402
3,3,105,2806
4,4,106,4444
...,...,...,...
151,151,349,7532
152,152,350,4190
153,153,351,473
154,154,992,952


In [ ]:
df_return_mon_sta

,반납 시간(M),반납 스테이션 번호,이용 횟수(월)
0,2017-01,0,3826
1,2017-01,101,109
2,2017-01,103,44
3,2017-01,104,89
4,2017-01,105,17
...,...,...,...
5683,2019-12,350,37
5684,2019-12,351,149
5685,2019-12,992,19
5686,2019-12,998,1


In [ ]:
df_return_to_sta

,이용 횟수(전체),총 이용 횟수(월)
0,0,1174956
1,101,9207
2,103,5121
3,104,9330
4,105,2696
...,...,...
153,350,3662
154,351,406
155,992,795
156,998,21
